![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [153]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV

# Import any additional modules and start coding below

# Preprocesamiento

## Importamos el dataset

In [154]:
df = pd.read_csv('rental_info.csv')

## Observamos el dataset

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


In [156]:
df['special_features'].head(5)

0    {Trailers,"Behind the Scenes"}
1    {Trailers,"Behind the Scenes"}
2    {Trailers,"Behind the Scenes"}
3    {Trailers,"Behind the Scenes"}
4    {Trailers,"Behind the Scenes"}
Name: special_features, dtype: object

In [157]:
df.head(100)

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2005-07-06 17:07:30+00:00,2005-07-09 20:26:30+00:00,0.99,2006.0,0.99,123.0,12.99,"{Trailers,Commentaries}",0,0,1,0,0.9801,15129.0,0.9801
96,2005-07-28 05:16:51+00:00,2005-08-02 04:45:51+00:00,1.99,2006.0,0.99,123.0,12.99,"{Trailers,Commentaries}",0,0,1,0,3.9601,15129.0,0.9801
97,2005-08-23 02:49:23+00:00,2005-09-01 07:50:23+00:00,6.99,2006.0,0.99,123.0,12.99,"{Trailers,Commentaries}",0,0,1,0,48.8601,15129.0,0.9801
98,2005-05-29 01:23:02+00:00,2005-06-02 21:39:02+00:00,1.99,2006.0,0.99,123.0,12.99,"{Trailers,Commentaries}",0,0,1,0,3.9601,15129.0,0.9801


## Preprocesamos las columnas

In [158]:
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])

In [159]:
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days

In [160]:
df['deleted_scenes'] = df['special_features'].apply(lambda x: 1 if "Deleted Scenes" in x else 0)
df['behind_the_scenes'] = df['special_features'].apply(lambda x: 1 if "Behind the Scenes" in x else 0)

In [161]:
df_clean = df.drop(['rental_date', 'return_date', 'special_features'], axis=1)

## Dividimos las características del objetivo

In [162]:
X = df_clean.drop(['rental_length_days'], axis = 1)
y = df_clean['rental_length_days']

## Dividimos en entrenamiento y test con 0.2 de tamaño

In [163]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [164]:
X_train.columns

Index(['amount', 'release_year', 'rental_rate', 'length', 'replacement_cost',
       'NC-17', 'PG', 'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2',
       'deleted_scenes', 'behind_the_scenes'],
      dtype='object')

## Modelos a probar

In [165]:
regr = RandomForestRegressor(random_state=9)
legr  = LinearRegression()
GBR = GradientBoostingClassifier(random_state=9)
SVR = SVR()
lasso = Lasso(random_state=9)

## Divide las columnas númericas y las categóricas para poder apli

In [166]:
num_cols = ['amount', 'release_year', 'rental_rate', 'length', 'replacement_cost',
       'amount_2', 'length_2', 'rental_rate_2']
cat_cols = ['NC-17', 'PG', 'PG-13', 'R', 'deleted_scenes', 'behind_the_scenes']

scaler = StandardScaler()

In [167]:
transformer = ColumnTransformer([('cat_cols', 'passthrough', cat_cols),
                                ('num_cols', scaler, num_cols)])

In [168]:
pipe_legr = Pipeline([("preprocessing", transformer),
                ("classifier", legr)])
pipe_regr = Pipeline([("preprocessing", transformer),
                ("classifier", regr)])
pipe_SVR = Pipeline([("preprocessing", transformer),
                ("classifier", SVR)])
pipe_lasso = Pipeline([("preprocessing", transformer),
                ("classifier", lasso)])
pipe_gbr = Pipeline([("preprocessing", transformer),
                ("classifier", GBR)])

In [169]:
pipelines = [pipe_legr, pipe_regr, pipe_SVR, pipe_lasso, pipe_gbr]
mse_scores = [] 
for pipeline in pipelines:
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mse_scores.append(mse)

In [170]:
best_mse = min(mse_scores)
best_model_index = mse_scores.index(best_mse)
best_model = pipelines[best_model_index].steps[-1][1]

print("Best Model:", best_model)
print("Best MSE:", best_mse)

Best Model: RandomForestRegressor(random_state=9)
Best MSE: 2.0311721583371525
